### This is Exploratory Data Analysis for Key Metrics on the Logs

The exploration will follow the following steps:
1. Load test and training dataset and necessary python packages

2. Convert data in the three memory columns to analyzable form

3. Clean data points with "NA"

** 1. Load dataset and necessary python packages **

In [37]:
# Load python packages
import pandas as pd
import numpy as np

In [38]:
# Load csv file
logs = pd.read_csv('/Users/shanxing/Documents/GitHub/Autodesk_AWS_Spark_ETL_Log_Analysis/Dataset/Spark_ETL_Loginfo_Key_Metrics_Transformed_v2.csv')

** 2. Convert data in the three memory columns to analyzable form **

In [39]:
# Display dataset
logs.head()

,S.No.,Log File,state_accept_epoch (s),state_running_time,state_running_epoch (s),Waiting Time (in Seconds),spark_driver_start_time,Driver Memory,Executor Memory,Memory Store Capacity,Job Result
0,1,application_1513303661803_25646_asrd.cp.big.da...,NaN,NA,NA,NA,NA,NA,NA,1983.0MB,NA
1,2,application_1513303661803_23926_asrd.cp.big.da...,1.513792e+09,17/12/20 17:52:06,1513792326,6,17/12/20 17:51:57,12g,6144M,8.4GB,Job completed successfully
2,3,application_1513303661803_21767_asrd.cp.big.da...,NaN,NA,NA,NA,NA,NA,NA,143.6MB,NA
3,4,application_1513303661803_24558_asrd.cp.big.da...,1.513814e+09,17/12/21 00:01:41,1513814501,5,17/12/21 00:01:33,8G,15G,5.5GB,Job completed successfully
4,5,application_1513303661803_24282_asrd.cp.big.da...,1.513797e+09,17/12/20 19:05:06,1513796706,9,17/12/20 19:04:52,12G,16G,8.4GB,Job completed successfully


In [40]:
# to check all kinds of formats of Megabyte and Gigabyte
print(logs['Driver Memory'].str.extract('(.*\d)(\w*)', expand=True)[1].value_counts())
print(logs['Executor Memory'].str.extract('(.*\d)(\w*)', expand=True)[1].value_counts())
print(logs['Memory Store Capacity'].str.extract('(.*\d)(\w*)', expand=True)[1].value_counts())

g    324
G    175
M     44
Name: 1, dtype: int64
G    292
M    251
Name: 1, dtype: int64
GB    875
MB    156
Name: 1, dtype: int64


In [41]:
# Convert all Gigabyte into Megabyte
def GBtoMB(df):
    df = df.copy()
    num = df.str.extract('(.*\d)(\w*)', expand=True)[0]
    unit = df.str.extract('(.*\d)(\w*)', expand=True)[1]
    ifGB = unit.isin(['GB','G','g'])
    df[ifGB] = round(num[ifGB].astype(float)*1024,1)
    ifMB = unit.isin(['MB','M'])
    df[ifMB] = round(num[ifMB].astype(float),1)
    return df

logs['Driver Memory'] = GBtoMB(logs['Driver Memory']).replace('NA','')
logs['Executor Memory'] = GBtoMB(logs['Executor Memory'])
logs['Memory Store Capacity'] = GBtoMB(logs['Memory Store Capacity'])

In [42]:
# Rename changed memory columns
name_change = {'Driver Memory':'Driver Memory (MB)', 'Executor Memory':'Executor Memory (MB)', 'Memory Store Capacity':'Memory Store Capacity (MB)'}
logs = logs.rename(columns = name_change)

In [43]:
# Display changed memory columns
logs[['Driver Memory (MB)','Executor Memory (MB)','Memory Store Capacity (MB)']].head()

,Driver Memory (MB),Executor Memory (MB),Memory Store Capacity (MB)
0,NA,NA,1983
1,12288,6144,8601.6
2,NA,NA,143.6
3,8192,15360,5632
4,12288,16384,8601.6


**3. Clean data points with "NA"**

In [44]:
# Display value count of job result
logs['Job Result'].value_counts()

 NA                            516
 Job completed successfully    511
 Job failed                     30
Name: Job Result, dtype: int64

In [45]:
# remove 'NA'
jobs = logs[logs['Job Result'].isin([' Job failed',' Job completed successfully'])]
jobs = jobs.replace(' NA', np.nan)
jobs.head()

,S.No.,Log File,state_accept_epoch (s),state_running_time,state_running_epoch (s),Waiting Time (in Seconds),spark_driver_start_time,Driver Memory (MB),Executor Memory (MB),Memory Store Capacity (MB),Job Result
1,2,application_1513303661803_23926_asrd.cp.big.da...,1.513792e+09,17/12/20 17:52:06,1513792326,6,17/12/20 17:51:57,12288.0,6144.0,8601.6,Job completed successfully
3,4,application_1513303661803_24558_asrd.cp.big.da...,1.513814e+09,17/12/21 00:01:41,1513814501,5,17/12/21 00:01:33,8192.0,15360.0,5632.0,Job completed successfully
4,5,application_1513303661803_24282_asrd.cp.big.da...,1.513797e+09,17/12/20 19:05:06,1513796706,9,17/12/20 19:04:52,12288.0,16384.0,8601.6,Job completed successfully
5,6,application_1513303661803_21050_asrd.cp.big.da...,1.513730e+09,17/12/20 00:38:46,1513730326,5,17/12/20 00:38:37,2048.0,10240.0,1247.6,Job completed successfully
7,8,application_1513303661803_23378_asrd.cp.big.da...,1.513786e+09,17/12/20 16:11:25,1513786285,5,17/12/20 16:11:17,12288.0,10240.0,8601.6,Job completed successfully


In [46]:
# Display info of jobs
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541 entries, 1 to 1055
Data columns (total 11 columns):
S.No.                         541 non-null int64
Log File                      541 non-null object
 state_accept_epoch (s)       538 non-null float64
 state_running_time           537 non-null object
 state_running_epoch (s)      537 non-null object
Waiting Time (in Seconds)     537 non-null object
spark_driver_start_time       541 non-null object
Driver Memory (MB)            541 non-null float64
Executor Memory (MB)          541 non-null float64
Memory Store Capacity (MB)    541 non-null float64
Job Result                    541 non-null object
dtypes: float64(4), int64(1), object(6)
memory usage: 50.7+ KB


In [47]:
# close look at missing values
jobs[jobs[' state_running_time'].isnull()]

,S.No.,Log File,state_accept_epoch (s),state_running_time,state_running_epoch (s),Waiting Time (in Seconds),spark_driver_start_time,Driver Memory (MB),Executor Memory (MB),Memory Store Capacity (MB),Job Result
106,107,application_1513303661803_18625_asrd.cp.big.da...,NaN,NaN,NaN,NaN,17/12/19 01:01:56,12288.0,10240.0,8601.6,Job failed
482,483,application_1513303661803_18626_asrd.cp.big.da...,NaN,NaN,NaN,NaN,17/12/19 01:01:58,12288.0,10240.0,8601.6,Job failed
483,484,application_1513303661803_18856_asrd.cp.big.da...,1.513650e+09,NaN,NaN,NaN,17/12/19 02:19:46,12288.0,12480.0,8601.6,Job failed
926,927,application_1513303661803_18632_asrd.cp.big.da...,NaN,NaN,NaN,NaN,17/12/19 01:02:08,12288.0,10240.0,8601.6,Job failed


**4. export a cleaned csv file**

In [48]:
jobs.to_csv('/Users/shanxing/Documents/GitHub/Autodesk_AWS_Spark_ETL_Log_Analysis/Dataset/cleaned_jobs.csv')